# Stage 0: SETUP
The following libraries are used directly. For the full list of isntalled ppackages and versions, please see requuirements.txt

In [1]:

# For processing
import pandas as pd
import numpy as np

# Stage 1: Data Acquisition

Data is downloaded as csv files, and is already available in this repository in the data folder. See the readme for details on the source of the data.

## Page Data
Page data is downloaded from [this](https://figshare.com/articles/dataset/Untitled_Item/5513449) repository.

In [6]:
page_data = pd.read_csv('data/raw/page_data.csv')
page_data.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


## Population Data
Population Data is downloaded from [this](https://docs.google.com/spreadsheets/d/1CFJO2zna2No5KqNm9rPK5PCACoXKzb-nycJFhV689Iw/edit#gid=283125346) google doc.

In [7]:
pop_data = pd.read_csv('data/raw/WPDS_2020_data.csv')
pop_data.head()

,FIPS,Name,Type,TimeFrame,Data (M),Population
0,WORLD,WORLD,World,2019,7772.850,7772850000
1,AFRICA,AFRICA,Sub-Region,2019,1337.918,1337918000
2,NORTHERN AFRICA,NORTHERN AFRICA,Sub-Region,2019,244.344,244344000
3,DZ,Algeria,Country,2019,44.357,44357000
4,EG,Egypt,Country,2019,100.803,100803000


# Stage 2: Data Processing
In this stage we combine and clean the data, and use the [ORES](https://github.com/wikimedia/ores) client to get predicted article quality.

# Clean and combine
Remove the templates from page data and regions from pop data

In [37]:
page_data_clean = page_data.loc[~page_data["page"].str.contains("^Template"), :]
pop_data_clean = pop_data.loc[pop_data["Type"] == "Country", :]

# Left join to keep countries without articles
combined_data = pop_data_clean.merge(page_data_clean, how="outer", left_on="Name", right_on="country")

# Record unmatched countries
unmatched_pop_data = combined_data[combined_data["page"].isna()].drop(columns=page_data_clean.columns)
unmatched_pop_data.to_csv('data/unmatched/wp_wpds_countries-no_match.csv')
print("{} Countries could not be matched".format(len(unmatched_pop_data)))

# Record unmatched pages
unmatched_page_data = combined_data[combined_data["Name"].isna()].drop(columns=pop_data_clean.columns)
unmatched_page_data.to_csv('data/unmatched/page_data-no_match.csv')
print("{} Pages could not be matched".format(len(unmatched_page_data)))

# Clean
combined_data_complete = combined_data.dropna()

26 Countries could not be matched
1859 Pages could not be matched


In [38]:
unmatched_page_data["country"].unique()

array(['Czech Republic', 'Salvadoran', 'Rhodesian', 'Congo, Dem. Rep. of',
       'East Timorese', 'Faroese', 'Cape Colony', 'South Korean',
       'Samoan', 'Montserratian', 'Pitcairn Islands',
       'Saint Kitts and Nevis', 'Macedonia', 'Abkhazia', 'Niuean',
       'Ivorian', 'Carniolan', 'Saint Lucian', 'South African Republic',
       'Hondura', 'Incan', 'Chechen', 'Jersey', 'Guernsey',
       'Saint Vincent and the Grenadines', 'South Ossetian',
       'Cook Island', 'Omani', 'Tokelauan', 'Swaziland', 'Dagestani',
       'Greenlandic', 'Ossetian', 'Palauan', 'Somaliland', 'Rojava'],
      dtype=object)